# Data Modeling

In [602]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
import seaborn as sns 
import missingno as msno
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler , OneHotEncoder,PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector,ColumnTransformer
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn import set_config

In [603]:
ames_train=pd.read_csv('../datasets/clean_ames_train.csv')
ames_test=pd.read_csv('../datasets/clean_ames_test.csv')

In [604]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [605]:
ames_train.shape
ames_test.shape
#checking column types to see which ones will need OneHotEncoder

(878, 41)

In [606]:
ames_train.corr()[['saleprice']].sort_values(by="saleprice",ascending=False)

,saleprice
saleprice,1.000000
overall_qual,0.800207
exter_qual,0.712146
gr_liv_area,0.697038
kitchen_qual,0.692336
garage_area,0.649897
garage_cars,0.647781
total_bsmt_sf,0.629303
1st_flr_sf,0.618486
bsmt_qual,0.612188


In [660]:
X=ames_train[['id','overall_qual','exter_qual',	
'gr_liv_area',
'kitchen_qual',
'garage_area',
'garage_cars',
'total_bsmt_sf',
'1st_flr_sf',
'bsmt_qual',
'year_built',
'garage_finish',
'year_remod/add',
'fireplace_qu',
'full_bath',
'totrms_abvgrd',
'mas_vnr_area',
'fireplaces',
'bsmtfin_sf_1',
'bsmtfin_type_1',
'open_porch_sf',
'wood_deck_sf']]

y=ames_train["saleprice"]

In [661]:
ames_test=ames_test[['id','overall_qual','exter_qual',	
'gr_liv_area',
'kitchen_qual',
'garage_area',
'garage_cars',
'total_bsmt_sf',
'1st_flr_sf',
'bsmt_qual',
'year_built',
'garage_finish',
'year_remod/add',
'fireplace_qu',
'full_bath',
'totrms_abvgrd',
'mas_vnr_area',
'fireplaces',
'bsmtfin_sf_1',
'bsmtfin_type_1',
'open_porch_sf',
'wood_deck_sf']]

In [662]:
X_train, X_test, y_train, y_test= train_test_split(X,y,random_state=42)

In [663]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [664]:
print(linreg.score(X_train, y_train))
print(linreg.score(X_test, y_test))

0.8143775822860749
0.8689338228107979


In [672]:
predictions = linreg.predict(ames_test)
ames_test['SalePrice'] = predictions
submissions = ames_test[['id', 'SalePrice']]
submissions.to_csv('../datasets/predictions/first_prediction.csv', index=False)

C:\Users\adoni\anaconda3\lib\site-packages\sklearn\base.py:484: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- SalePrice
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 23 features, but LinearRegression is expecting 22 features as input.

In [671]:
# Set Index
submission.set_index('Id', inplace=True) 

KeyError: "None of ['Id'] are in the columns"

In [668]:
train=pd.read_csv('../datasets/train.csv')
test=pd.read_csv('../datasets/test.csv')

In [627]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [623]:
ohe = OneHotEncoder(sparse=False, drop='if_binary', handle_unknown='ignore')

In [638]:
smart_encoder = make_column_transformer(
    (ohe, ['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 
'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 
'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 
'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond', 'Foundation', 
'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 
'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 
'Functional', 'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature', 
]), # These are all of the categorical features
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [640]:
ames_train_encoded = smart_encoder.fit_transform(train)

In [645]:
ridge=Ridge()